In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from node2vec import Node2Vec
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from ipynb.fs.full.test_train_split import mask_test_edges
import numpy as np
from scipy import sparse

In [2]:
foursquare_edgelist = pd.read_csv("../datasets/foursquare_edgelist.csv")

In [3]:
foursquare_edgelist.shape

(196270, 4)

In [4]:
#generating the graph
net = nx.Graph()

for i in range(len(foursquare_edgelist)):
    net.add_edge(foursquare_edgelist.iloc[i]['user_id'], foursquare_edgelist.iloc[i]['user2'])
    net.add_edge(foursquare_edgelist.iloc[i]['user_id'], foursquare_edgelist.iloc[i]['venue_id'])

In [5]:
net.number_of_nodes()

19638

In [6]:
net.number_of_edges()

42465

In [7]:
net.nodes()

NodeView((123141, 134107, '4840fe6bf964a52030501fe3', '49f76cc2f964a5209d6c1fe3', '4da7cf1981541df437af6cf7', '4ad8795ff964a520a21121e3', '40f1d480f964a5206a0a1fe3', '4858e403f964a520bf501fe3', '4ca6342076d3a093b22fff6a', '3fd66200f964a52005e71ee3', '4569264af964a520de3d1fe3', '3fd66200f964a5200ce41ee3', '4197f180f964a520111e1fe3', '4e498e05aeb7de71b38c15ff', '45b9eeecf964a520db411fe3', '3fd66200f964a520f6e41ee3', '4aca9125f964a52060c220e3', '4c7e5ec4d598a093dd83c562', '4b53cfe9f964a52025ac27e3', '4d10f5c038bb6ea8969bc7aa', '4c2af1fa8ef52d7f95a930ba', '4afefbd9f964a5207a3222e3', '4cf406ab7bf3b60ca2176e7f', '4e66ceadaeb7e985aa7b65a4', '4fd49307e4b0b5691947492c', '4c1573e1a1010f47841d4e18', '50ad97c2e4b06b4737016a40', '4d766d8f46a8b60cb0ee372a', '4dcdba448877851243e6347f', '4c53be6b06901b8d4da0a34a', '50f6a5f0e4b017ec5540b9b5', '50227b55e4b079961002339f', 168005, 1428411, '4b1d4b75f964a520560e24e3', '49d18dfdf964a5208f5b1fe3', '45ebc90ef964a5208f431fe3', '4a9c60eef964a520ff3620e3', '4b06

In [54]:
# predict whether there is a link between
node_index_map = dict()
for index, node in enumerate(net.nodes()):
    node_index_map[node] = index

In [ ]:
#generating an adjacency list and storing 
# adj_list = nx.generate_adjlist(net)

# node_check = []

# for line in adj_list:
#     token = line.split()
#     node_check.append(token[0])

## Train Test split

In [8]:
np.random.seed(0) # make sure train-test split is consistent between notebooks
adj_sparse = nx.adjacency_matrix(net)

# Perform train-test split
adj_train, train_edges, train_edges_false, val_edges, val_edges_false, \
    test_edges, test_edges_false = mask_test_edges(adj_sparse, test_frac=.3, val_frac=.1)

In [9]:
print(adj_sparse)

  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	1
  (0, 10)	1
  (0, 11)	1
  (0, 12)	1
  (0, 13)	1
  (0, 14)	1
  (0, 15)	1
  (0, 16)	1
  (0, 17)	1
  (0, 18)	1
  (0, 19)	1
  (0, 20)	1
  (0, 21)	1
  (0, 22)	1
  (0, 23)	1
  (0, 24)	1
  (0, 25)	1
  :	:
  (19617, 17066)	1
  (19617, 19618)	1
  (19618, 19617)	1
  (19619, 19620)	1
  (19619, 19621)	1
  (19620, 19619)	1
  (19621, 19619)	1
  (19622, 16320)	1
  (19622, 19623)	1
  (19623, 19622)	1
  (19624, 6082)	1
  (19625, 18525)	1
  (19626, 18525)	1
  (19627, 12012)	1
  (19628, 18668)	1
  (19629, 18668)	1
  (19630, 18045)	1
  (19631, 18045)	1
  (19632, 15339)	1
  (19632, 19633)	1
  (19633, 19632)	1
  (19634, 19227)	1
  (19635, 14907)	1
  (19636, 14907)	1
  (19637, 14907)	1


In [10]:
#saving adjacency matrix for future use
sparse.save_npz("adj_train_matrix.npz", adj_train)

In [11]:
np.savetxt('train_edges.txt', train_edges, fmt='%d')
np.savetxt('train_edges_false.txt', train_edges_false, fmt='%d')
np.savetxt('val_edges.txt', val_edges, fmt='%d')
np.savetxt('val_edges_false.txt', val_edges_false, fmt='%d')
np.savetxt('test_edges.txt', test_edges, fmt='%d')
np.savetxt('test_edges_false.txt', test_edges_false, fmt='%d')

In [12]:
#loading the adjacency matrix
adj_train = sparse.load_npz("adj_train_matrix.npz")

In [13]:
train_edges = np.loadtxt('train_edges.txt', dtype=int)
train_edges_false = np.loadtxt('train_edges_false.txt', dtype=int)
val_edges = np.loadtxt('val_edges.txt', dtype=int)
val_edges_false = np.loadtxt('val_edges_false.txt', dtype=int)
test_edges = np.loadtxt('test_edges.txt', dtype=int)
test_edges_false = np.loadtxt('test_edges_false.txt', dtype=int)

## Node2vec

In [14]:
# generating graph from the train data
g_train = nx.from_scipy_sparse_matrix(adj_train)

In [15]:
# fitting the graph using the node2vec model
node2vec = Node2Vec(g_train, dimensions=128, walk_length=80, num_walks=10, workers=4, p=0.5, q=6)

Computing transition probabilities: 100%|██████████| 19638/19638 [00:16<00:00, 1208.62it/s]


In [16]:
model = node2vec.fit(window=10, min_count=1)

In [17]:
#generating the mappings
emb_mappings = model.wv

In [18]:
# Create node embeddings matrix (rows = nodes, columns = embedding features)
emb_list = []
for node_index in range(0, adj_train.shape[0]):
    node_str = str(node_index)
    node_emb = emb_mappings[node_str]
    emb_list.append(node_emb)
emb_matrix = np.vstack(emb_list)

In [30]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Input: positive test/val edges, negative test/val edges, edge score matrix
# Output: ROC AUC score, ROC Curve (FPR, TPR, Thresholds), AP score
def get_roc_score(edges_pos, edges_neg, score_matrix, apply_sigmoid=False):

    # Edge case
    if len(edges_pos) == 0 or len(edges_neg) == 0:
        return (None, None, None)

    # Store positive edge predictions, actual values
    preds_pos = []
    pos = []
    for edge in edges_pos:
        if apply_sigmoid == True:
            preds_pos.append(sigmoid(score_matrix[edge[0], edge[1]]))
        else:
            preds_pos.append(score_matrix[edge[0], edge[1]])
        pos.append(1) # actual value (1 for positive)
        
#     print("prediction_pos:", preds_pos)

    # Store negative edge predictions, actual values
    preds_neg = []
    neg = []
    for edge in edges_neg:
        if apply_sigmoid == True:
            preds_neg.append(sigmoid(score_matrix[edge[0], edge[1]]))
        else:
            preds_neg.append(score_matrix[edge[0], edge[1]])
        neg.append(0) # actual value (0 for negative)
        
#     print("prediction_neg:", preds_neg)
        
    # Calculate scores
    preds_all = np.hstack([preds_pos, preds_neg])
    labels_all = np.hstack([np.ones(len(preds_pos)), np.zeros(len(preds_neg))])
    
#     print("prediction:", preds_all)
#     print("labels:", labels_all)
    roc_score = roc_auc_score(labels_all, preds_all)
    # roc_curve_tuple = roc_curve(labels_all, preds_all)
    ap_score = average_precision_score(labels_all, preds_all)
    
    # return roc_score, roc_curve_tuple, ap_score
    return roc_score, ap_score

In [31]:
#Normal dot product score
score_matrix = np.dot(emb_matrix, emb_matrix.T)

# Val set scores
if len(val_edges) > 0:
    n2v_val_roc, n2v_val_ap = get_roc_score(val_edges, val_edges_false, score_matrix, apply_sigmoid=True)
else:
    n2v_val_roc = None
    n2v_val_roc_curve = None
    n2v_val_ap = None
        
# Test set scores
n2v_test_roc, n2v_test_ap = get_roc_score(test_edges, test_edges_false, score_matrix, apply_sigmoid=True)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


prediction_pos: [1.0, 0.9999361246431011, 0.9956447634999723, 0.9922586032631814, 0.9999999946080806, 0.9979486327702447, 0.9999999988801074, 0.9995805400836043, 0.9981813938930768, 0.9999999999999165, 0.9999956486593132, 0.9971750590924229, 0.999935632844166, 4.5799954529232644e-06, 8.823236947410596e-08, 0.9999999970950422, 0.9358810107072402, 0.9976047040650877, 0.9999999791081078, 0.7730762085876247, 0.9999999999999991, 5.436500558568578e-05, 0.9950305520816176, 0.9999994100201163, 0.9999999762692872, 0.9999999973590539, 0.9999634023667797, 0.999608553306378, 0.9999993515140431, 1.0, 0.9830115362241882, 0.9999999999982645, 0.6638961747338623, 0.02020093824197186, 0.9999999773467062, 1.7443828087902898e-14, 1.2939572439325333e-06, 0.7934744765173101, 1.0, 0.9999999999999931, 0.1144915986072454, 0.9479177708870873, 0.034810885519429076, 0.9999995607811283, 4.8669354190939955e-05, 1.0, 4.3953648240711275e-07, 0.9997358533446418, 1.0, 0.9999999999810107, 0.999915760187989, 0.9849033416

TypeError: 'NoneType' object is not iterable

In [21]:
print("test_roc:",n2v_test_roc)
print("val_roc:",n2v_val_roc)

test_roc: 0.6238004185316262
val_roc: 0.6281509803690997


As this is binary classification threshold for probability is 0.5 ie > 0.5 predicted as 1 and <= 0.5 predicted as 0

In [55]:
node1 = node_index_map[1386757]
node2 = node_index_map[1446783]
print("node1:"+str(node1)+" node2:"+str(node2))

node1:10396 node2:10397


In [53]:
# predicted correctly that user 1386757 is a friend of 1446783
sigmoid(score_matrix[10396,10397])

0.9999999999999105

In [59]:
net.edges(1386757)

EdgeDataView([(1386757, 537615), (1386757, 1446783), (1386757, '4ad507bff964a520660121e3')])

In [63]:
# predicting incorrectly that 134107 visited 4840fe6bf964a52030501fe3 which is not true
node1 = node_index_map[134107]
node2 = node_index_map['4840fe6bf964a52030501fe3']
print("node1:"+str(node1)+" node2:"+str(node2))

node1:1 node2:2


In [64]:
sigmoid(score_matrix[1,2])

0.6518003046642535

In [65]:
net.edges(134107)

EdgeDataView([(134107, 123141), (134107, 208065), (134107, '43a52546f964a520532c1fe3'), (134107, 873751), (134107, '519a94e3498e722d3d9ae1bf')])